In [1]:
import numpy as np
import pandas as pd


In [2]:
cocacola = pd.read_excel("D:\\Assignment__Rstudio\\Forecasting\\CocaCola_Sales_Rawdata.xlsx")

In [3]:
cocacola.head()

,Quarter,Sales
0,Q1_86,1734.827000
1,Q2_86,2244.960999
2,Q3_86,2533.804993
3,Q4_86,2154.962997
4,Q1_87,1547.818996


In [4]:
quarter = ['Q1','Q2','Q3','Q4']

In [5]:
n = cocacola['Quarter'][0]

In [6]:
n[0:2]

'Q1'

In [7]:
cocacola['quarter']=0

In [8]:

for i in range(42):
    n=cocacola['Quarter'][i]
    cocacola['quarter'][i]=n[0:2]

C:\Users\TEMP.DESKTOP-LD1KGA9.028\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\TEMP.DESKTOP-LD1KGA9.028\Anaconda3\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [9]:

dummy = pd.DataFrame(pd.get_dummies(cocacola['quarter']))

In [10]:
coco = pd.concat((cocacola,dummy),axis=1)

In [11]:

t = np.arange(1,43)

In [12]:

coco['t']=t

In [13]:

coco['t_square']=coco['t']*coco['t']

In [14]:
log_Sales = np.log(coco['Sales'])
coco['log_Sales']=log_Sales

In [15]:
train = coco.head(38)
test = coco.tail(4)

In [16]:
coco.Sales.plot()

In [17]:
import statsmodels.formula.api as smf

# Linear Model


In [18]:
linear = smf.ols('Sales~t',data=train).fit()
predlin = pd.Series(linear.predict(pd.DataFrame(test['t'])))
rmselin = np.sqrt((np.mean(np.array(test['Sales'])-np.array(predlin))**2))
rmselin

421.17878760022813

# Quadratic Model


In [19]:
quad = smf.ols('Sales~t+t_square',data=train).fit()

predquad = pd.Series(quad.predict(pd.DataFrame(test[['t','t_square']])))

rmsequad = np.sqrt(np.mean((np.array(test['Sales'])-np.array(predquad))**2))

rmsequad

475.56183518315095

# Exponential Model

In [20]:
expo = smf.ols('log_Sales~t',data=train).fit()

predexp = pd.Series(expo.predict(pd.DataFrame(test['t'])))

predexp

38    8.365398
39    8.389143
40    8.412888
41    8.436633
dtype: float64

In [22]:
rmseexpo = np.sqrt(np.mean((np.array(test['Sales'])-np.array(np.exp(predexp)))**2))

rmseexpo

466.24797310672346

# Additive seasonality

In [24]:
additive = smf.ols('Sales~ Q1+Q2+Q3+Q4',data = train).fit()

predadd = pd.Series(additive.predict(pd.DataFrame(test[['Q1','Q2','Q3','Q4']])))

predadd




38    2938.434220
39    2712.930886
40    2472.555199
41    3150.000098
dtype: float64

In [26]:
rmseadd = np.sqrt(np.mean((np.array(test['Sales'])-np.array(predadd))**2))

rmseadd

1860.0238154547283

# 
Additive seasonality with linear trend

In [27]:

addlinear = smf.ols('Sales~t+Q1+Q2+Q3+Q4',data=train).fit()

predaddlinear = pd.Series(addlinear.predict(pd.DataFrame(test[['t','Q1','Q2','Q3','Q4']])))

predaddlinear

38    4292.265126
39    4066.761792
40    3961.769195
41    4639.214094
dtype: float64

In [28]:
rmseaddlinear = np.sqrt(np.mean((np.array(test['Sales'])-np.array(predaddlinear))**2))

rmseaddlinear

464.98290239822427

# Additive seasonality with quadratic trend

In [29]:
addquad = smf.ols('Sales~t+t_square+Q1+Q2+Q3+Q4',data=train).fit()

predaddquad = pd.Series(addquad.predict(pd.DataFrame(test[['t','t_square','Q1','Q2','Q3','Q4']])))

predaddquad

38    4852.677797
39    4709.186074
40    4638.364982
41    5406.022652
dtype: float64

In [30]:
rmseaddquad = np.sqrt(np.mean((np.array(test['Sales'])-np.array(predaddquad))**2))

rmseaddquad

301.73800719352977

# multiplicative seasonality

In [31]:
mulsea = smf.ols('log_Sales~Q1+Q2+Q3+Q4',data=train).fit()

predmul = pd.Series(mulsea.predict(pd.DataFrame(test[['t','t_square','Q1','Q2','Q3','Q4']])))

rmsemul = np.sqrt(np.mean((np.array(test['Sales'])-np.array(np.exp(predmul)))**2))

rmsemul

1963.3896400779709

# Multiplicative seasonality with linear trend

In [32]:
mullin = smf.ols('log_Sales~t+Q1+Q2+Q3+Q4',data=train).fit()

predmullin = pd.Series(mullin.predict(pd.DataFrame(test[['t','Q1','Q2','Q3','Q4']])))

rmsemulin = np.sqrt(np.mean((np.array(test['Sales'])-np.array(np.exp(predmullin)))**2))

rmsemulin

225.5243904982721

# Multliplicative seasonality with quadratic trend

In [33]:


mul_quad = smf.ols('log_Sales~t+t_square+Q1+Q2+Q3+Q4',data=train).fit()

pred_mul_quad = pd.Series(mul_quad.predict(test[['t','t_square','Q1','Q2','Q3','Q4']]))

rmse_mul_quad = np.sqrt(np.mean((np.array(test['Sales'])-np.array(np.exp(pred_mul_quad)))**2))

rmse_mul_quad

581.8457187971785

# Tabulating the rmse values



In [34]:
data = {'Model':pd.Series(['rmse_mul_quad','rmseadd','rmseaddlinear','rmseaddquad','rmseexpo','rmselin','rmsemul','rmsemulin','rmsequad']),'Values':pd.Series([rmse_mul_quad,rmseadd,rmseaddlinear,rmseaddquad,rmseexpo,rmselin,rmsemul,rmsemulin,rmsequad])}

data

{'Model': 0    rmse_mul_quad
 1          rmseadd
 2    rmseaddlinear
 3      rmseaddquad
 4         rmseexpo
 5          rmselin
 6          rmsemul
 7        rmsemulin
 8         rmsequad
 dtype: object, 'Values': 0     581.845719
 1    1860.023815
 2     464.982902
 3     301.738007
 4     466.247973
 5     421.178788
 6    1963.389640
 7     225.524390
 8     475.561835
 dtype: float64}

In [35]:

Rmse = pd.DataFrame(data)
Rmse

,Model,Values
0,rmse_mul_quad,581.845719
1,rmseadd,1860.023815
2,rmseaddlinear,464.982902
3,rmseaddquad,301.738007
4,rmseexpo,466.247973
5,rmselin,421.178788
6,rmsemul,1963.389640
7,rmsemulin,225.524390
8,rmsequad,475.561835
